Black Scholes Model in Python for Predicting Options Premiums

Team members: Javier Arevalo, Anson Tsang


-A stock option is the right to buy or sell a stock at an agreed price and date, each option contracts represents 100 shares of that stock.

The two types of options used for different situations are either:
1. calls: betting a stock will increase in value
2. puts: betting a stock will decrease in value 


There are also many different styles of options, such as the following:
1. American options can be exercised before the expiration date
2. European options can only be exercised at the expiration date. 


There are many different methods for pricing options, but the Black Scholes model is considered to be one of the best ways of determining fair prices of options. 

It requires five variables:

-K: the strike price of an option

-S: the current stock price

-t: time to expiration

-r: risk free rate

-$\sigma$: volatility measured as the standard deviation of the log return


The PDE form of the Black Scholes equation is:

$\frac{δV}{δt}+\frac{1}{2}σ^2S^2\frac{δ^2V}{δS^2}+rS\frac{δV}{δS}-rV=0$

Where V represents the value of the option, S the value of the underlying asset.

It's possible to use a finite difference scheme to solve this PDE numerically:

$\frac{V^{i+1}_j - V^i_j}{Δt}+\frac{1}{2}σ^2S^2(\frac{V_{j+1}^i-2V_j^i+2_{j-1}^i}{Δx^2})+rS\frac{V_{j+1}^i-V_{j-1}^i}{2Δx}-rSV_{j}^i = 0$

Rearranging the equation, and given $S \equiv jΔx$:

$V_j^{i+1}= {α}C_{j-1}^n + {β}C_j^n +{ρ}C_{j+1}^n$

$α = \frac{σ^2j^2Δt - rjΔt}{2}$

$β = 1 - σ^2j^2Δt -rΔt$

$ρ = \frac{σ^2j^2Δt + rjΔt}{2}$

where i represents time discretization, and j represents price ('spatial') discretization.

With understanding of initial and boundary conditions, the Black Scholes equation can be solved using numerical techniques for different derivatives, such as American options. 

However, closed form exact solutions can be developed for special cases, such as the European call/put options, which can only be exercised at expiration:

$\begin{cases}
V(0,t) = 0 \space{}|\space{} t \ge 0&(1)\\
V(S,t) = S \space{}|\space{}S→∞&(2)\\
V(S,T) = max(S-K,0) &(3)\\
\end{cases}$

The first boundary condition states the derivative has no value if S=0 is at expiry, which holds for all $t\ge0$. The second derivative assumes that as the stock price approaches infinity, the price of the option will become the stock price S. The final boundary condition provides the payoff of the derivative at maturity time T, which is the max of either the difference between the stock price and strike price, or 0.

To solve the Black Scholes equation analytically, the PDE must be converted to a known form, the classical 1D heat diffusion equation with Dirchlet boundary conditions:

$\frac{δu}{δτ}=\frac{δ^2u}{δx^2}$

Where temperature u is a function of time $τ$ and spatial location x.

The closed form solution of the heat equation is:

$u(x,τ)=u_o*Φ(x,t)$

Which is the initial value convoluted by the fundamental solution to the heat equation:

$u(x,τ)=\frac{1}{4τ\sqrt{πτ}}∫_{-∞}^{∞}u_0(s)e^{\frac{-(x-s)^2}{4τ}}ds$

Using different mathemetical and substitution techniques, the closed form solution of Black Scholes PDE is:

$d_{1}=log(\frac{S}{K})+\frac{(r+σ^{2}/2)*T}{σ*\sqrt{T}}$ \
$d_{2} = d_1 - \sigma*\sqrt{T}$

$C(T) = SN(d_1)-Ke^{-rT}N(d_2)$\
$P(T)=Ke^{-rT}-S + C(T)$

There are two ways to calculate volatility, either using historical data or calculating the implied volatility through numerical root finding such as Newton-Raphson method. We will calculate the implied volatility at $C(t_i<T)$ and use it as an input parameter to estimate $C(t_j<T)$, where $t_i<t_j$. 

Assumptions made for this model:
1. It works on European options that can only be exercised at expiration
2. No dividends paid during the option's life 
3. No transaction costs and commission costs in buying the option
4. The returns on the underlying are normally distributed 

Closed Form Black Scholes in Python:

1. Need to calculate the probability of receiving the stock at the expiration of the option (price increasing more than strike price). This will be d1

2. Need to calculate the risk adjusted probability that the option will be exercised. This will be d2

Need a function to calculate d1 and d2. 

3. After that we could input them into a Normal distribution cumulative distribution function using scipy, which will give us the probabilities. 

Each requires 5 inputs: S, K, T, r and $σ$



In [ ]:
from math import log, sqrt, pi, exp
from scipy.stats import norm 
from datetime import datetime, date
import numpy as np
import pandas as pd 
from pandas import DataFrame

def d1(S, K, T, r, sigma):
  return (log(S/K) + (r+sigma**2/2.)*T)/(sigma*sqrt(T))

def d2(S, K, T, r, sigma):
  return d1(S, K, T, r, sigma) - sigma*sqrt(T)

Now we can implement the Black Scholes formula in Python using the values calculated from above functions. 

However, there has to be two different formulas: one for calls, and other for puts:


In [ ]:
def bs_call(S, K, T, r, sigma):
  return S*norm.cdf(d1(S, K, T, r, sigma)) - K*exp(-r*T)*norm.cdf(d2(S, K, T, r, sigma))

def bs_put(S, K, T, r, sigma):
  return K*exp(-r*T)-S+bs_call(S, K, T, r, sigma)

Collect data:

To test our model need to get historical data for a specific stock and other inputs related to the stock. 

For this example use Yahoo Finance and pandas library to get the necessary data 

*Are able to calculate sigma value (volatility of the stock) by multiplying the standard deviation of the stock returns over the past year by the square root of 252 (number of days market is open over a year). 

*For current price, use the last close price in dataset. 

*Input r, the risk free rate, as the 10-year US treasury yield which I can get from ^TNX

In [ ]:
from datetime import datetime, date
import numpy as np
import pandas as pd 
import pandas_datareader.data as web 

stock = 'SPY'
expiry = '12-18-2020'
strike_price = 370

today = datetime.now()
one_year_ago = today.replace(year=today.year - 1)

#This call is enough to call yahoo finance api or web scrape info? Double check
df = web.DataReader(stock, 'yahoo', one_year_ago, today)

df = df.sort_valus(by="Date")
df = df.dropna()
df = df.assign(close_day_before=df.Close.shift(1))
df['returns'] = ((df.Close - df.close_day_before)/df.close_day_before)


#Calculate sigma, or volatility of stock as follows:
sigma = np.sqrt(252) * df['returns'].std()
uty = (web.DataReader(
    "^TNX", 'yahoo', today.replace(day=today.day-1), today)['Close'].iloc[-1])/100
lcp = df['Close'].iloc[-1]
t = (datetime.strptime(expiry, "%m-%d-%Y") - datetime.utcnow()).days / 365

print('The Option Price is: ', bs_call(lcp, strike_price, t, uty, sigma))


RemoteDataError: ignored

Utilizing the function:

We can now output the price of the option by using the function created before. 

For this example, use a call option on 'SPY' with expiry '12-18-2020' and a strike price of '370'

If all done correctly should get a value of ~16.169. 
*Try with expiry date of 2020 instead of 2022 if an error or mismatch in values 

Newton Raphson's method of root finding is used to calculate implied volatility

$x_{n+1} = x_n - \frac{f(x_n)}{f'(x_n)}$

$f'(x_n)≈\frac{f(x_n + ϵ)-f(x_n - ϵ)}{2ϵ}$

In [ ]:
def cal_vol(S, K, T, r, V_temp, type): #V = value of put/call, type = 0 or 1 depending if put/call

  sig = 0.5 #Initial Guess for Volatility
  ep = 0.001 #finite difference approx
  tol = 1000

  while tol < 0.0001:
    if (V == 1):

      V_temp  = bs_call(S, K, T, r, sig)
      derivative = (bs_call(S, K, T, r, sig+ep) - bs_call(S, K, T, r, sig-ep))/(2*ep)
      sig_temp = sig
      sig = sig - V_temp/derivative
  
    else: 

      V_temp  = bs_put(S, K, T, r, sig)
      derivative = (bs_put(S, K, T, r, sig+ep) - bs_put(S, K, T, r, sig-ep))/(2*ep)
      sig_temp = sig
      sig = sig - V_temp/derivative

    tol = (sig_temp - sig)

  return sig
